# RNN Encoder-decoder

This is the notebook for constructing Theano RNN encoder-decoder. The text for an encoder-decoder can be found in a lot of places, and one reference is here: [http://anie.me/rnn-encoder-decoder/](http://anie.me/rnn-encoder-decoder/). This notebook might also contain a section on vanilla RNN classification task, and a section on RNN autoencoder as well. Note in this implementation, we are being a lot more efficient than some blog posts (like in WildML).

In [19]:
from classifier.rnn_layers import *
from classifier.util import *
from data.snli_util import *
from vis.util import *
import numpy as np

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data SNLI

We use SNLI for inference task (the main purpose of encoder-decoder). We use a partial IMDB dataset provided by Yoon Kim (also preprocessed into numpy form).

In [3]:
data = load_data('data/snli_processed.npz')

# NOTE: right now this doesn't work, but we want to hold it to 
# the same standard as other code
# Print out all the keys and values from the data dictionary
for k, v in data.iteritems():
  if type(v) == np.ndarray:
    print k, type(v), v.shape, v.dtype
  else:
    print k, type(v), len(v)

## Look at the Data

All the sentences in train, dev, and test are encoded already by indicies. Now we see if we can successfully decode them into actual sentence with our `decode()` function.

## Vanilla RNN stepforward

We implement the RNN for a single forward step in Theano

In [23]:
N, D, H = 3, 10, 4

x = np.asarray(np.linspace(-0.4, 0.7, num=N*D).reshape(N, D), dtype='float32')
prev_h = np.asarray(np.linspace(-0.2, 0.5, num=N*H).reshape(N, H), dtype='float32')
Wx = np.asarray(np.linspace(-0.1, 0.9, num=D*H).reshape(D, H), dtype='float32')
Wh = np.asarray(np.linspace(-0.3, 0.7, num=H*H).reshape(H, H), dtype='float32')
b = np.asarray(np.linspace(-0.2, 0.4, num=H), dtype='float32')

x_sym = T.matrix('x')
prev_h_shared = theano.shared(
            value=prev_h,
            name='prev_h',
            borrow=True
        )
Wx_shared = theano.shared(
            value=Wx,
            name='Wx',
            borrow=True
)
Wh_shared = theano.shared(
            value=Wh,
            name='Wh',
            borrow=True
)
b_shared = theano.shared(
            value=b,
            name='b',
            borrow=True
)

next_h = rnn_step_forward(x_sym, prev_h_shared, Wx_shared, Wh_shared, b_shared)

get_value_nexth = theano.function([x_sym], next_h)

expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

print 'next_h error: ', rel_error(expected_next_h, get_value_nexth(x))

 next_h error:  9.608901251e-08


## Vanilla RNN Forward

Now we use `theano.scan()` to implement RNN forward over T sequence.